In [5]:
# import sys
# sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

## Configuration

In [13]:
ROOT = ".."
INPUT = "input"
LISH_MOA = "lish-moa"
NUM_FOLD = 5
NUM_OPTUNA_TRIAL = 30
N_COMP_GENES = 50
N_COMP_CELLS = 15

In [7]:
def seed_everything(seed=334):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=334)

## Read data

In [8]:
train = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_features.csv"))
test = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "test_features.csv"))
train_targets_scored = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_targets_scored.csv"))
train_targets_nonscored = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_targets_nonscored.csv"))
sub = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "sample_submission.csv"))

In [10]:
GENES = [col for col in train.columns if col.startswith('g-')]
CELLS = [col for col in train.columns if col.startswith('c-')]

In [11]:
train_targets_scored.sum()[1:].sort_values()

atp-sensitive_potassium_channel_antagonist      1
erbb2_inhibitor                                 1
diuretic                                        6
autotaxin_inhibitor                             6
protein_phosphatase_inhibitor                   6
                                             ... 
serotonin_receptor_antagonist                 404
dopamine_receptor_antagonist                  424
cyclooxygenase_inhibitor                      435
proteasome_inhibitor                          726
nfkb_inhibitor                                832
Length: 206, dtype: object

## PCA features + Existing features

In [25]:
def make_pca_features(df_train: pd.DataFrame, df_test: pd.DataFrame, n_components: int, use_cols: list, gene_or_cell: str, cat_flg):
    data = pd.concat([pd.DataFrame(df_train[use_cols]), pd.DataFrame(df_test[use_cols])])
    data_pca = PCA(n_components=n_components, random_state=334).fit_transform(data[use_cols])

    train_pca = data_pca[:df_train.shape[0]]
    test_pca = data_pca[-df_test.shape[0]:]

    train_pca = pd.DataFrame(train_pca, columns=[f'pca_'+gene_or_cell+'-{i}' for i in range(n_components)])
    test_pca = pd.DataFrame(test_pca, columns=[f'pca_'+gene_or_cell+'-{i}' for i in range(n_components)])

    ret_df_train = pd.concat([df_train, train_pca], axis=1)
    ret_df_test = pd.concat([df_test, test_pca], axis=1)
    return ret_df_train, ret_df_test

In [ ]:
train, test = make_pca_features(train, test, N_COMP_GENES, GENES, 'G')